In [ ]:
# Is SC or wire disordered?

In [ ]:
try:
    # If this can be imported, it assumes you are on the TU Delft network with access to cluster
    from hpc05 import HPC05Client as Client
    import os
    os.environ['SSH_AUTH_SOCK'] = os.path.join(os.path.expanduser('~'), 'ssh-agent.socket')
    client = Client('basnijholt', timeout=30)
    print("Connected to hpc05")
except ImportError:
    from ipyparallel import Client
    client = Client(profile='python3')
    print("Connected to local engines")

In [ ]:
dview = client[:]
lview = client.load_balanced_view()
len(dview)

In [ ]:
%matplotlib inline
import holoviews as hv
hv.notebook_extension()
import sympy.interactive
sympy.interactive.init_printing('mathjax')

In [ ]:
# %%px --local

import kwant
from kwant.digest import uniform
import numpy as np
from itertools import product
import subprocess
import types
import os.path
import deepdish as dd
import matplotlib.pyplot as plt
from supercurrent import (current_at_phase,
                          discretized_hamiltonian,
                          hoppingkind_at_interface,
                          hoppingkind_in_shape,
                          make_params,
                          peierls,
                          SimpleNamespace)


def cylinder_sector(r1, r2=0, L=1, L0=0, phi=360, angle=0, a=10):
    """Returns the shape function and start coords.

    Parameters:
    -----------
    r1 : int
        Inner radius in nm.
    r2 : int
        Outer radius in nm.
    L : int
        Length of wire from L0 in nm, -1 if infinite in x-direction.
    L0 : int
        Start position in x.
    phi : int
        Coverage angle in degrees.
    angle : int
        Angle of tilting from top in degrees.
    a : int
        Discretization constant in nm.

    Returns:
    --------
    (shape_func, *(start_coords))
    """
    phi *= np.pi / 360
    angle *= np.pi / 180
    r1sq, r2sq = r1 ** 2, r2 ** 2

    def sector(pos):
        x, y, z = pos
        n = (y + 1j * z) * np.exp(1j * angle)
        y, z = n.real, n.imag
        rsq = y ** 2 + z ** 2

        shape_yz = r2sq <= rsq < r1sq and z >= np.cos(phi) * np.sqrt(rsq)
        return (shape_yz and L0 <= x < L) if L > 0 else shape_yz
    r_mid = (r1 + r2) / 2
    return sector, (L - a, r_mid * np.sin(angle), r_mid * np.cos(angle))


def make_3d_wire(a, L, r1, r2, phi, angle, L_sc, disorder,
                 with_vlead, with_leads, with_shell, spin, holes):
    """Creates a cylindrical 3D wire partially covered with a superconducting (SC) shell, 
    but without superconductor in the scattering region of length L.

    Default arguments:
    ------------------
    (a=10, L=50, r1=50, r2=70, phi=135, angle=0, disorder=False, 
     with_vlead=True, with_leads=True, L_sc=10, with_shell=True,
     spin=True, holes=True)
    Note: we are not using default parameter because I save them in a dictionary, to
    save to file.

    Parameters:
    -----------
    a : int
        Discretization constant in nm.
    L : int
        Length of wire (the scattering part without SC shell.) Should be bigger
        than 4 unit cells (4*a) to have the vleads in a region without a SC shell.
    r1 : int
        Radius of normal part of wire in nm.
    r2 : int
        Radius of superconductor in nm.
    phi : int
        Coverage angle of superconductor in degrees.
    angle : int
        Angle of tilting of superconductor from top in degrees.    
    disorder : bool
        When True, syst requires 'disorder' and 'salt' aguments.
    with_vlead : bool
        If True a SelfEnergyLead with zero energy is added to a slice of the system.
    with_leads : bool
        If True it appends infinite leads with superconducting shell.
    L_sc : int
        Number of unit cells that has a superconducting shell. If the system has
        infinite leads, set L_sc=a.
    with_shell : bool
        Adds shell the the correct areas. If False no SC shell is added and only
        a cylindrical wire will be created.

    Returns:
    --------
    syst : kwant.builder.FiniteSystem
        The finilized system.
    hopping : function
        Function that returns the hopping matrix between the two cross sections
        of where the SelfEnergyLead is attached.
    """
    assert L_sc % a == 0
    assert L % a == 0

    tb_normal, tb_sc, tb_interface = discretized_hamiltonian(a, spin, holes)
    lat = tb_normal.lattice
    syst = kwant.Builder()
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))

    # The parts with a SC shell are not counted in the length L, so it's
    # modified as:
    L += 2*L_sc

    # Wire scattering region shapes
    shape_normal = cylinder_sector(r1=r1, angle=angle, L=L, a=a)
    # Superconductor slice in the beginning of the scattering region of L_sc
    # unit cells
    shape_sc_start = cylinder_sector(
        r1=r2, r2=r1, phi=phi, angle=angle, L=L_sc, a=a)
    # Superconductor slice in the end of the scattering region of L_sc unit
    # cells
    shape_sc_end = cylinder_sector(
        r1=r2, r2=r1, phi=phi, angle=angle, L0=L-L_sc, L=L, a=a)

    # Lead shapes
    shape_sc_lead = cylinder_sector(
        r1=r2, r2=r1, phi=phi, angle=angle, L=-1, a=a)
    shape_normal_lead = cylinder_sector(r1=r1, angle=angle, L=-1, a=a)

    def onsite_dis(site, p):
        identity = np.eye(4) if spin and holes else np.eye(2)
        return p.disorder * (uniform(repr(site), repr(p.salt)) - 0.5) * identity

    # Add onsite terms in the scattering region
    syst[lat.shape(*shape_normal)] = lambda s, p: tb_normal.onsite(s, p) + (onsite_dis(s, p) if disorder else 0)

    if with_shell:
        syst[lat.shape(*shape_sc_start)] = tb_sc.onsite
        syst[lat.shape(*shape_sc_end)] = tb_sc.onsite

    # Add onsite terms in the infinite lead
    lead[lat.shape(*shape_normal_lead)] = tb_normal.onsite
    if with_shell:
        lead[lat.shape(*shape_sc_lead)] = tb_sc.onsite

    for hop, func in tb_normal.hoppings.items():
        # Add hoppings in normal parts of wire and lead with Peierls
        # substitution
        ind = np.argmax(hop.delta)  # Index of direction of hopping
        syst[hoppingkind_in_shape(hop, shape_normal, syst)] = peierls(func, ind, a)
        lead[hoppingkind_in_shape(hop, shape_normal_lead, lead)] = peierls(func, ind, a)

    if with_shell:
        for hop, func in tb_sc.hoppings.items():
            # Add hoppings in superconducting parts of wire and lead
            syst[hoppingkind_in_shape(hop, shape_sc_start, syst)] = func
            syst[hoppingkind_in_shape(hop, shape_sc_end, syst)] = func
            lead[hoppingkind_in_shape(hop, shape_sc_lead, lead)] = func

        for hop, func in tb_interface.hoppings.items():
            # Add hoppings at the interface of superconducting parts and normal
            # parts of wire and lead
            syst[hoppingkind_at_interface(
                hop, shape_sc_start, shape_normal, syst)] = func
            syst[hoppingkind_at_interface(
                hop, shape_sc_end, shape_normal, syst)] = func
            lead[hoppingkind_at_interface(
                hop, shape_sc_lead, shape_normal_lead, lead)] = func

    def cut(x_cut):
        """Return the sites at a cross section at x_cut."""
        sites = [lat(x, y, z)
                 for x, y, z in (i.tag for i in syst.sites()) if x == x_cut]
        return sorted(sites, key=lambda s: s.pos[2] * 10000 + s.pos[1])

    # Define left and right cut in wire in the middle of the wire, a region
    # without superconducting shell.
    l_cut = cut(L // (2*a) - 1)
    r_cut = cut(L // (2*a))
    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(
        lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)

    if with_vlead:
        syst.leads.append(vlead)
    if with_leads:
        syst.attach_lead(lead)
        syst.attach_lead(lead.reversed())

    syst = syst.finalized()

    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]

    def hopping(syst, args=()):
        """Function that returns the hopping matrix of the electrons
        between the two cross sections."""
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping


def save_data(fname, p, constants, Bs, tol, syst_params, T, current_phase, **kwargs):
    if os.path.exists(fname):
        return "File already existed"

    def SimpleNamespace_save(p):
        """Removes functions from SimpleNamespace."""
        p_new = {key: val for key, val in p.__dict__.items()
                 if not isinstance(val, types.FunctionType)}
        return p_new

    def get_git_revision_hash():
        return subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode("utf-8").replace('\n', '')

    data = {'p': SimpleNamespace_save(p),
            'constants': SimpleNamespace_save(constants),
            'B_x': Bs,
            'tol': tol,
            'syst_params': syst_params,
            'T': T,
            'current_phase': current_phase,
            'git_hash': get_git_revision_hash(), **kwargs}

    dd.io.save(fname, data, compression='blosc')


def file_name(base, hamiltonian_params, syst_params, T):
    fname = '{}__L_{}_nm__mu_{:.0f}_meV__a_{:.0f}_nm__T_{}_K__{}{}{}{}.h5'
    leads = "infinite_leads" if syst_params[
        'with_leads'] else 'leads_{}_nm'.format(syst_params['L_sc'])
    orbital_str = '__no_orbital' if not hamiltonian_params['orbital'] else ''
    zeeman_str = '__no_zeeman' if hamiltonian_params['g'] == 0 else ''
    alpha_str = '__no_spinorbit' if hamiltonian_params['alpha'] == 0 else ''
    fname = fname.format(base, syst_params['L'], hamiltonian_params['mu'],
                         syst_params['a'], T, leads, orbital_str, zeeman_str, alpha_str)
    return fname


# Creating some wires as example

In [ ]:
# Create system with infinite leads
syst_params = dict(a=10, L=50, r1=50, r2=70, phi=135, angle=0, holes=True, spin=True,
                   disorder=False, with_vlead=True, with_leads=True, L_sc=10, with_shell=True)
syst, hopping = make_3d_wire(**syst_params)
kwant.plot(syst)

In [ ]:
# Create system without leads
syst_params['with_leads'] = False
syst, hopping = make_3d_wire(**syst_params)
kwant.plot(syst)

In [ ]:
# Create system without leads and bigger SC regions
syst_params['L_sc'] = 30
syst, hopping = make_3d_wire(**syst_params)
kwant.plot(syst)

# $T$ vs $I_c$ with disorder

In [ ]:
%%px --local
p = make_params(Delta=60, alpha=20, mu=50, B_x=0, B_y=0, t_interface=7/8*constants.t, disorder=None, salt=None)
syst_params = dict(a=8, L=400, r1=50, r2=70, phi=135, angle=0, disorder=True, with_shell=True, 
                   with_vlead=True, with_leads=False, L_sc=400, spin=True, holes=True)

syst, hopping = make_3d_wire(**syst_params)
tol = 1e-2

In [ ]:
Ts = [0.1, 0.5, 1]
salts = range(10)
disorders = [0, 10, 20, 40]
I = lview.map_async(lambda x: I_c(syst, hopping, p.update(salt=x[0], disorder=x[1]), T=x[2], tol=tol),
                    product(salts, disorders, Ts))

I.wait_interactive()

Ic = np.array(I.result()).reshape(len(salts), len(disorders), len(Ts), -1)

I_cs, phases = Ic.T

In [ ]:
wire_type = "infinite_leads" if syst_params['with_leads'] else 'leads_{}_nm'.format(syst_params['L_sc'])

fname = 'T_vs_Ic__L_{}_nm__mu_{:.0f}_meV__{}__a_{:.0f}_nm.h5'.format(syst_params['L'], 
                                                                     p.mu,
                                                                     syst_params['a'],
                                                                     wire_type)

save_data(fname, p, constants, None, tol, syst_params, Ts, I_cs, phases,
          disorders=disorders, salts=salts)

# Mean free path

In [ ]:
%%px --local
def transmission(syst, p):
    smatrix = kwant.smatrix(syst, args=[p])
    return smatrix.transmission(0, 1), smatrix.num_propagating(0)

p = make_params(Delta=0, alpha=20, mu=50, B_x=0, B_y=0, B_z=0, disorder=None, salt=None)
syst_params = dict(a=8, L=None, r1=50, r2=70, phi=135, angle=0, disorder=True,
                   with_vlead=False, with_leads=True, L_sc=8, with_shell=False, holes=False, spin=True)

In [ ]:
Ls = np.arange(80, 500, 80)
conductances = []
num_propagating = []
for L in Ls:
    print(L)
    dview.execute('syst_params["L"] = {}'.format(L))
    %px syst, hopping = make_3d_wire(**syst_params)
    salts = range(30)
    disorders = np.arange(0, 150, 10)
    conductance = lview.map_async(lambda x: transmission(syst, p.update(salt=x[0], disorder=x[1])), 
                        product(salts, disorders))
    conductance.wait_interactive()
    conductance = np.array(conductance.result()).reshape(len(salts), len(disorders), -1)
    num_propagating.append(conductance[:, :, 1])
    conductances.append(conductance[:, :, 0])

In [ ]:
save_data('data/mfp__mu_{:.0f}_meV__a_{:.0f}_nm.h5'.format(p.mu, syst_params['a']), p, constants, None, None, syst_params, 
          T=None, I_cs=None, phases=None, conductances=conductances, salts=salts,
          disorders=disorders, Ls=Ls, num_propagating=num_propagating)

# Disorder $B_x$ vs $I_c$

In [ ]:
%%px --local

hamiltonian_params = dict(Delta=60, mu=50, B_x=None, B_y=0,
                          t_interface=7/8*constants.t, disorder=None, salt=None,
                          orbital=True, g=50)
syst_params = dict(a=8, L=None, r1=50, r2=70, phi=135, angle=0, disorder=True, 
                   with_vlead=True, with_leads=None, L_sc=None, with_shell=True, holes=True, spin=True)

tol = 1e-2

In [ ]:
Bs = np.linspace(0, 1, 100)
salts = range(30)
disorders = [0, 1, 2, 5, 10, 20, 30, 40, 50]

for syst_params['L'] in [80, 160, 320, 640]:
    for T in [0.1, 0.5, 1]:
        for syst_params['with_leads'], syst_params['L_sc'] in [(True, 8), (False, 400)]:
            fname = file_name('disorder_B_vs_I_c', hamiltonian_params, syst_params, T)
            if not os.path.isfile(fname):
                dview['syst_params'] = syst_params
                dview['hamiltonian_params'] = hamiltonian_params
                dview['T'] = T
                %px p = make_params(**hamiltonian_params)
                %px syst, hopping = make_3d_wire(**syst_params)
                I = lview.map_async(lambda x: I_c(syst, hopping, p.update(salt=x[0], disorder=x[1], B_x=x[2]), T, tol), 
                                    product(salts, disorders, Bs))
                I.wait_interactive()
                current_phase = I.result()
                save_data(fname, make_params(**hamiltonian_params), constants, Bs, tol,
                          syst_params, T, current_phase, disorders=disorders, salts=salts)

# No disorder $B_x$ vs $I_c$:

In [ ]:
%%px --local 

syst_params = dict(a=8, r1=50, r2=70, phi=135, angle=0,
                   disorder=False, with_vlead=True,
                   with_shell=True, holes=True, spin=True,
                   L=None, with_leads=None, L_sc=None,)

hamiltonian_params = dict(Delta=60, t_interface=7/8*constants.t, 
                          mu=None, B_x=None, alpha=None, g=None, orbital=None)
tol = 1e-2
Bs = np.linspace(0, 1, 100)
max_frequencies = 200

In [ ]:
T_list = [0.1, 0.5, 1]
orbital_list = [True, False]
g_list = [0, 50]
alpha_list = [0, 20]
mu_list = [30, 60]

for L in [80, 160, 320, 640]:
    for with_leads, L_sc in [(True, 8), (False, 400)]:
        syst_params['L'] = L
        syst_params['with_leads'] = with_leads
        syst_params['L_sc'] = L_sc
        fname = 'no_disorder_B_vs_I_c__L_{}_nm{}.h5'.format(L, '__infinite_leads' if with_leads else '')
        if not os.path.isfile(fname):
            print(fname)
            dview['syst_params'] = syst_params
            %px p = make_params(**hamiltonian_params)
            %px syst, hopping = make_3d_wire(**syst_params)
            vals = list(product(T_list, Bs, orbital_list, g_list, alpha_list, mu_list))
            I = lview.map_async(lambda x: I_c(syst, hopping, p.update(B_x=x[1], orbital=x[2], g=x[3], alpha=x[4], mu=x[5]),
                                              x[0], tol, max_frequencies), vals)
            I.wait_interactive()
            current_phase = I.result()
            save_data(fname, make_params(**hamiltonian_params), constants,
                      Bs, tol, syst_params, None, current_phase, 
                      max_frequencies=max_frequencies, vals=vals, 
                      T_list=T_list, orbital_list=orbital_list, 
                      g_list=g_list, alpha_list=alpha_list, mu_list=mu_list)

# Induced gap

In [ ]:
%%px --local
# Create system with infinite leads
syst_params = dict(a=8, L=800, r1=50, r2=70, phi=135, angle=0, 
                   disorder=False, with_vlead=True, with_leads=True, L_sc=400)
syst, hopping = make_3d_wire(**syst_params)
lead = syst.leads[1]

p = make_params(t_interface=7/8*constants.t)

def evs(lead, p):
    h0 = lead.cell_hamiltonian(args=[p])
    t0 = lead.inter_cell_hopping(args=[p])
    ham = h0 + t0 + t0.conj().T
    ev = np.linalg.eigvalsh(ham)
    return np.abs(ev).min()

In [ ]:
mus = np.linspace(2, 20, 50)
deltas = np.linspace(0, 100, 50)
vals = list(product(deltas, mus))
dview.scatter('vals', vals, block=True)
%px res = [evs(lead, p) for p.Delta, p.mu in vals]
res = np.reshape(dview.gather('res', block=True), (len(deltas), len(mus)))
print("Max gap is {} meV".format(max(res.min(axis=1))))

In [ ]:
hv.Curve((deltas, res.min(axis=1)), kdims=['$\Delta$'], vdims=['$E_{gap}$'], label="$E_{gap}$ over range of $\Delta$'s")

In [ ]:
Delta_ind = {delta: hv.Path((mus, evs), kdims=['$\mu$', ('E_gap', '$E_{gap}$')]) 
      for evs, delta in zip(res, deltas)}

hm = hv.HoloMap(Delta_ind, kdims=['$\Delta$'])
(hm.select(E_gap=(0.2, 0.3)) * hv.HLine(0.25))

# Gate

In [ ]:
# Plot test
syst_params = dict(a=8, L=160, r1=50, r2=70, phi=135, angle=0, holes=True, spin=True,
                   disorder=False, with_vlead=True, with_leads=True, L_sc=8, with_shell=True)
syst, hopping = make_3d_wire(**syst_params)
gate_fun = gate(syst, 100, 160)
sites = [gate_fun(pos[0]) for pos in [i.pos for i in syst.sites]]
kwant.plot(syst, site_lw=0, site_color=sites, colorbar=False)

In [ ]:
%%px --local 

def find_nearest(array, value):
    idx = np.abs(np.array(array) - value).argmin()
    return array[idx]

def gate(syst, V, gate_size):
    x_positions = sorted(set(i.pos[0] for i in syst.sites))
    x_mid = (max(x_positions) - min(x_positions)) / 2
    x_L = find_nearest(x_positions, x_mid - gate_size / 2)
    x_R = find_nearest(x_positions, x_mid + gate_size / 2)
    return lambda x: V if x > x_L and x <= x_R else 0

syst_params = dict(a=8, r1=50, r2=70, phi=135, angle=0,
                   disorder=False, with_vlead=True,
                   with_shell=True, holes=True, spin=True,
                   L=640, with_leads=True, L_sc=8)

hamiltonian_params = dict(Delta=60, t_interface=7/8*constants.t, 
                          mu=50, B_x=None, alpha=20, g=50, orbital=True)
tol = 1e-2
Bs = np.linspace(0, 1, 100)
Vs = np.linspace(0, 40, 100)
max_frequencies = 200
T = 1
gate_size = None

p = make_params(**hamiltonian_params)
syst, hopping = make_3d_wire(**syst_params)

In [ ]:
vals = list(product(Bs, Vs))

for gate_size in [80, 160, 320, 640]:
    dview['gate_size'] = gate_size
    I = lview.map_async(lambda x: I_c(syst, hopping, p.update(B_x=x[0], V=gate(syst, x[1], gate_size)),
                                      T, tol, max_frequencies), vals)
    I.wait_interactive()
    current_phase = I.result()
    fname = "current_as_function_of_gate_and_B_x__gate_size_{}_nm.h5".format(gate_size)
    save_data(fname, make_params(**hamiltonian_params), constants,
              Bs, tol, syst_params, T, current_phase, 
              max_frequencies=max_frequencies, vals=vals, Vs=Vs, gate_size=gate_size)